In [ ]:
import sys
from pathlib import Path

_BASE_DIR = Path().resolve().parent.parent
print("(!) Make sure this dir is project directory: ", _BASE_DIR)
sys.path.append(str(_BASE_DIR))

from utils.settings import get_in_out_dirs

_LAB_NAME = "lab9"

INPUT_DIR, OUTPUT_DIR = get_in_out_dirs(base_name=_LAB_NAME)

DATA_DIR = INPUT_DIR / "data_1a"
DATA_DIR.mkdir(parents=True, exist_ok=True)

TRAIN_DIR = DATA_DIR / "train"
TRAIN_DIR.mkdir(parents=True, exist_ok=True)

TEST_DIR = DATA_DIR / "test"
TEST_DIR.mkdir(parents=True, exist_ok=True)

(!) Make sure this dir is project directory:  /home/ars/Desktop/university/bsu-nn


# Лаб. 9

## Задание 1.  (из  темы  9.1):

### 1А 
Дообучите нейронную сеть VGG16 для распознавания изображений только двух классов. Используйте дообученную сеть для классификации своих изображений.


1. Скачаем картинки
2. Они будут в одной папке **train_transformed**, переложим их в две папки **cats**, **dogs**
3. Рандомно выберем 10 на тест

In [ ]:
import shutil
import kagglehub

path = kagglehub.dataset_download("abhinavnayak/catsvdogs-transformed")
print("Dataset path:", path)

# Копируем в твой TRAIN_DIR
for item in Path(path).iterdir():
    dest = TRAIN_DIR / item.name
    if item.is_dir():
        shutil.copytree(item, dest, dirs_exist_ok=True)
    else:
        shutil.copy(item, dest)

Dataset path: /home/ars/.cache/kagglehub/datasets/abhinavnayak/catsvdogs-transformed/versions/1
Готово! Тренировочные данные в: /home/ars/Desktop/university/bsu-nn/data/inputs/lab9/data/train_classes
Тестовые данные в: /home/ars/Desktop/university/bsu-nn/data/inputs/lab9/data/test_classes


In [ ]:
src = TRAIN_DIR / "train_transformed"
cats = TRAIN_DIR / "cats"
dogs = TRAIN_DIR / "dogs"

cats.mkdir(parents=True, exist_ok=True)
dogs.mkdir(parents=True, exist_ok=True)

for f in src.glob("*.jpg"):
    name = f.name.lower()
    if name.startswith("cat"):
        shutil.move(str(f), cats / f.name)
    elif name.startswith("dog"):
        shutil.move(str(f), dogs / f.name)

print("Картинки разложены по папкам:", cats, dogs)

shutil.rmtree(TRAIN_DIR / "train_transformed")

Картинки разложены по папкам: /home/ars/Desktop/university/bsu-nn/data/inputs/lab9/data/train_classes/cats /home/ars/Desktop/university/bsu-nn/data/inputs/lab9/data/train_classes/dogs


In [ ]:
import shutil
import random

# Создаём тестовую выборку (10 случайных из каждого класса)
for cls in ["cats", "dogs"]:
    src_folder = TRAIN_DIR / cls
    test_folder = TEST_DIR / cls
    test_folder.mkdir(parents=True, exist_ok=True)
    
    files = list(src_folder.glob("*.jpg"))
    sample = random.sample(files, min(10, len(files)))
    
    for f in sample:
        shutil.move(str(f), test_folder / f.name)

print("Готово! Тренировочные данные в:", TRAIN_DIR)
print("Тестовые данные в:", TEST_DIR)

Готово! Тренировочные данные в: /home/ars/Desktop/university/bsu-nn/data/inputs/lab9/data/train_classes
Тестовые данные в: /home/ars/Desktop/university/bsu-nn/data/inputs/lab9/data/test_classes


***ТЕПЕРЬ САМА МОДЕЛЬ***

In [ ]:
from tensorflow.keras.applications import VGG16
from tensorflow.keras import layers, models
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.metrics import BinaryAccuracy

# -----------------------------
# 1. Базовая сеть без верхушки
# -----------------------------
target_size_tp = (128,128)
base_model = VGG16(weights="imagenet", include_top=False, input_shape=(*target_size_tp, 3))
base_model.trainable = False  # сначала замораживаем

# -----------------------------
# 2. Модель "голова"
# -----------------------------
model = models.Sequential([
    base_model,
    layers.Flatten(),
    layers.Dense(256, activation="relu"),
    layers.Dropout(0.5),
    layers.Dense(1, activation="sigmoid")  # 2 класса → сигмоида
])

model.compile(optimizer="adam", loss="binary_crossentropy", metrics=[BinaryAccuracy()])

# -----------------------------
# 3. Генератор данных
# -----------------------------
datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2)
batch_size = 16
train_gen = datagen.flow_from_directory(
    TRAIN_DIR,
    target_size=target_size_tp,
    batch_size=batch_size,
    class_mode="binary",
    subset="training",
    shuffle=True
)

val_gen = datagen.flow_from_directory(
    TRAIN_DIR,
    target_size=target_size_tp,
    batch_size=batch_size,
    class_mode="binary",
    subset="validation",
    shuffle=False
)



Found 1584 images belonging to 2 classes.
Found 396 images belonging to 2 classes.


In [39]:
MODEL_DIR = OUTPUT_DIR / "model"
MODEL_DIR.mkdir(parents=True, exist_ok=True)

OUT_MODEL_PATH = MODEL_DIR / "vgg16_cats_dogs.h5"

In [ ]:
from tensorflow.keras.models import load_model

# -----------------------------
# 4. Тренировка
# -----------------------------

if OUT_MODEL_PATH.exists():
    print("Модель найдена, загружаем вместо тренировки...")
    model = load_model(OUT_MODEL_PATH)

else:
    print("Модель не найдена, тренируем...")
    history = model.fit(
        train_gen,
        validation_data=val_gen,
        epochs=1
    )
    print("Сохраняем модель...")
    model.save(OUT_MODEL_PATH)


Модель найдена, загружаем вместо тренировки...


In [ ]:
# -----------------------------
# 5. Тест
# -----------------------------

test_datagen = ImageDataGenerator(rescale=1./255)

test_gen = test_datagen.flow_from_directory(
    TEST_DIR,
    target_size=(128,128),      # должно совпадать с размером, который использовала модель
    batch_size=16,
    class_mode="binary",        # два класса
    shuffle=False               # важно, чтобы предсказания шли в том же порядке, что и файлы
)

# Оценка модели на тесте
loss, acc = model.evaluate(
    test_gen
)
print(f"Test loss: {loss:.4f}, accuracy: {acc:.4f}")

Found 20 images belonging to 2 classes.


/home/ars/Desktop/university/bsu-nn/.venv/lib/python3.12/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


2/2 ━━━━━━━━━━━━━━━━━━━━ 2s 271ms/step - accuracy: 0.9000 - loss: 0.2245
Test loss: 0.2245, accuracy: 0.9000


### 1Б
Дообучите нейронную сеть VGG16 для распознавания изображений только трех классов. Используйте дообученную сеть для классификации своих изображений.

In [ ]:
# -----------------------------
# 1. Папки данных
# -----------------------------
DATA_DIR_2 = INPUT_DIR / "data_1b"
DATA_DIR_2.mkdir(parents=True, exist_ok=True)

DATA_DIR_2_SRC = INPUT_DIR / "data_1b_src"
DATA_DIR_2_SRC.mkdir(parents=True, exist_ok=True)

TRAIN_DIR_2 = DATA_DIR_2 / "train"
TRAIN_DIR_2.mkdir(parents=True, exist_ok=True)

TEST_DIR_2 = DATA_DIR_2 / "test"
TEST_DIR_2.mkdir(parents=True, exist_ok=True)

In [50]:
# -----------------------------
# 2. Скачиваем и копируем датасет
# -----------------------------

for item in DATA_DIR_2_SRC.iterdir():
    if item.is_dir() and item.name.endswith("_train"):
        cls_name = item.name.replace("_train","")
        dest = TRAIN_DIR_2 / cls_name
        shutil.copytree(item, dest, dirs_exist_ok=True)
    
    elif item.is_dir() and item.name.endswith("_test"):
        cls_name = item.name.replace("_test","")
        dest = TEST_DIR_2 / cls_name
        shutil.copytree(item, dest, dirs_exist_ok=True)

In [53]:
# -----------------------------
# 3. Архитектура модели
# -----------------------------
from tensorflow.keras.applications import VGG16
from tensorflow.keras import layers, models
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.metrics import CategoricalAccuracy

target_size_tp = (128,128)
batch_size = 16

base_model = VGG16(weights="imagenet", include_top=False, input_shape=(*target_size_tp,3))
base_model.trainable = False

model = models.Sequential([
    base_model,
    layers.Flatten(),
    layers.Dense(256, activation="relu"),
    layers.Dropout(0.5),
    layers.Dense(3, activation="softmax")  # 3 класса
])

model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=[CategoricalAccuracy()])

In [54]:
# -----------------------------
# 4. Генераторы данных
# -----------------------------
datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2)

train_gen = datagen.flow_from_directory(
    TRAIN_DIR_2,
    target_size=target_size_tp,
    batch_size=batch_size,
    class_mode="categorical",  # для >2 классов
    subset="training",
    shuffle=True
)

val_gen = datagen.flow_from_directory(
    TRAIN_DIR_2,
    target_size=target_size_tp,
    batch_size=batch_size,
    class_mode="categorical",
    subset="validation",
    shuffle=False
)

test_datagen = ImageDataGenerator(rescale=1./255)
test_gen = test_datagen.flow_from_directory(
    TEST_DIR_2,
    target_size=target_size_tp,
    batch_size=batch_size,
    class_mode="categorical",
    shuffle=False
)

Found 72 images belonging to 3 classes.
Found 18 images belonging to 3 classes.
Found 60 images belonging to 3 classes.


In [55]:
# -----------------------------
# 5. Обучение или загрузка
# -----------------------------

MODEL_DIR = OUTPUT_DIR / "model"
MODEL_DIR.mkdir(parents=True, exist_ok=True)

OUT_MODEL_PATH_2 = MODEL_DIR / "vgg16_vehicles.h5"


if OUT_MODEL_PATH_2.exists():
    print("Модель найдена, загружаем вместо тренировки...")
    model = load_model(OUT_MODEL_PATH_2)

else:
    print("Модель не найдена, тренируем...")
    history = model.fit(
        train_gen,
        validation_data=val_gen,
        epochs=1
    )
    print("Сохраняем модель...")
    model.save(OUT_MODEL_PATH_2)

Модель не найдена, тренируем...


/home/ars/Desktop/university/bsu-nn/.venv/lib/python3.12/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


5/5 ━━━━━━━━━━━━━━━━━━━━ 8s 1s/step - categorical_accuracy: 0.5417 - loss: 1.6618 - val_categorical_accuracy: 0.6667 - val_loss: 1.2338


Сохраняем модель...


In [56]:
# -----------------------------
# 6. Оценка модели на тесте
# -----------------------------
loss, acc = model.evaluate(
    test_gen
)
print(f"Test loss: {loss:.4f}, accuracy: {acc:.4f}")

4/4 ━━━━━━━━━━━━━━━━━━━━ 3s 819ms/step - categorical_accuracy: 0.6667 - loss: 1.0603
Test loss: 1.0603, accuracy: 0.6667
